In [1]:
import math
import numpy as np
import pandas as pd
import scipy.stats as st
from tqdm.notebook import tqdm

## "All of Statistics: A Concise Course in Statistical Inference" 

### Chapter 10: Hypothesis Testing and p-values

**Problem 7** In 1861, 10 essays appeared in the *New Orleans Daily Crescent*. They  were  signed by "Quintus Curtius  Snodgrass" and some people suspected they were actually written by Mark Twain. To investigate this, we will consider the proportion of three letter words found in an author's work. From eight Twain essays we have:

.225 .262 .217 .240 .230 .229 .235 .217

From 10 Sodgrass essays we have :

.209 .205 .196 .210 .202 .207 .224 .223 .220 .201

(a) Perform a Wald test for equality of the means. Use the nonparametric plug-in estimator. Report the p-value and the 95 percent confidence interval for the difference in means. What do you conclude?

(b) Now use permutation test to avoid the use of large sample methods. What is your conclusion? (Brinegar (1963))

In [2]:
twain = np.array([.225, .262, .217, .240, .230, .229, .235, .217])
snodgrass  =  np.array([.209, .205, .196, .210, .202, .207, .224, .223, .220, .201])

In [3]:
# Nonparametric plug-in estimator simply the difference in the sample means.

est = twain.mean() - snodgrass.mean() # plug-in estimator (difference of sample means)

se = np.sqrt(twain.var(ddof=1)/twain.size + snodgrass.var(ddof=1)/snodgrass.size) # estimated standard error ussing sample variances

W = abs(est / se)

print(f'Twain mean = {twain.mean()}')
print(f'Snodgrasss mean ={snodgrass.mean()}')
print(f'Difference ={twain.mean() - snodgrass.mean()}')
print(f'p-value = {2*st.norm.cdf(-W):.4f}')
print(f'95% confidence interval = ({est-1.96*se:.3f}, {est+1.96*se:.3f})')

Twain mean = 0.231875
Snodgrasss mean =0.2097
Difference =0.022175
p-value = 0.0002
95% confidence interval = (0.010, 0.034)


There is a statistically very significant difference between the sample means. There seems to be a measurable difference in the writing style, although the books could still be written by the same person, just in a different style. We would need to control for other factors like the subject of the essays.

In [4]:
# Permutation test

rng = np.random.default_rng()

est = twain.mean() - snodgrass.mean()

pooled = np.concatenate([twain, snodgrass])

count = 0

reps = 10**6

for _ in range(reps): # Better to use the vectorized versions below
    permuted = rng.permuted(pooled)
    count += abs(permuted[:twain.size].mean() - permuted[twain.size:].mean()) >= est
    
print(count / reps)

0.000779


In [5]:
# Vectorized version (much faster!)

reindex = np.random.rand(10**6, pooled.size).argsort(axis=1) # A bit of a wasteful hack but faster in this example

(np.abs(pooled[reindex][:,:twain.size].mean(axis=1) - pooled[reindex][:,twain.size:].mean(axis=1)) > est).mean()

0.000721

In [6]:
# Another vectorized version that it better for longer arrays (but not quite as fast for this length 18 array)
# For longer arrays the argsort hack is costly because it needs to (implicitly) sort all the arrays

reps = 10**6

permutations = rng.permuted(np.stack((pooled,)*reps), axis=-1)

(np.abs(permutations[:,:twain.size].mean(axis=1) - permutations[:,twain.size:].mean(axis=1)) > est).mean()

0.000726

In [7]:
# Avoiding the count operation in the loop already speeds things up quite a bit

reps = 10**6
permutations = []
for _ in range(reps): # Avoiding the count increase in the loop makes it much faster
    permutations.append(rng.permuted(pooled))

permutations_array = np.array(permutations)

(np.abs(permutations_array[:,:twain.size].mean(axis=1) - permutations_array[:,twain.size:].mean(axis=1)) > est).mean()

0.000709

The p-value from the permutation test is around 0.0007, over three times larger than the p-value from the Wald test above, but still very small.

**Problem 10** Here are the number of elderly Jewish and Chinese women who died just before and after the Chinese Harvest Moon Festival.

Week | Chinese | Jewish
-- | -- | --
-2 | 55 | 141
-1 | 33 | 145
1 | 70 | 139
2 | 49 | 161

Compare the two mortality patterns. (Phillips and Smith (1990))

In [8]:
chinese = np.array([55, 33, 70, 49])
jewish = np.array([141, 145, 139, 161])

In [9]:
# Compare the deaths in weeks -1 and 1

before = chinese[1]
total = chinese[[1, 2]].sum()

est = before / total
se = np.sqrt(est * (1 - est) / total)

2 * st.norm.cdf(-abs(est - 1/2) / se) # p-value

9.365476186680414e-05

In [10]:
# Compare the deaths in weeks -1 and 1

before = jewish[1]
total = jewish[[1, 2]].sum()

est = before / total
se = np.sqrt(est * (1 - est) / total)

2 * st.norm.cdf(-abs(est - 1/2) / se) # p-value

0.7217552078897358

In [11]:
# Compare the deaths in weeks  -2 and 2

before = chinese[0]
total = chinese[[0, 3]].sum()

est = before / total
se = np.sqrt(est * (1 - est) / total)

2 * st.norm.cdf(-abs(est - 1/2) / se) # p-value

0.5556399329877082

In [12]:
# Compare the deaths in weeks  -2 and 2

before = jewish[0]
total = jewish[[0, 3]].sum()

est = before / total
se = np.sqrt(est * (1 - est) / total)

2 * st.norm.cdf(-abs(est - 1/2) / se)  # p-value

0.24874511916003872

The only significant difference was between weeks -1 and 1 in chinese women. That difference was very significant with a p-value of 0.00009 and a significant effect too.

**Problem 11** A randomized, double-blind experiment was conducted to assess the effectiveness of several drugs for reducing postoperative nausea. The data are as follows.

| | Number of patients | Incidence of Nausea
-- | -- | --
Placebo | 80 | 45
Chlorpromazine | 75 | 26
Dimenhydrinate | 85 | 52
Pentobarbital (100 mg) | 67 | 35
Pentobarbital (150 mg) | 85 | 37

(a) Test each drug versus the placebo at the 5 per cent level. Also, report the estimated odds-ratios. Summarize your findings.

(b) Use the Bonferroni and the FDR method to adjust for multiple testing. (Beecher (1959))

We can model the incidence of nausea for each group as a binomial random variable. The null hypothesis is that the distributions are equal, i.e. have the same $p$ parameter.

- The book probably expects us to use the Wald test with $\theta = p_1 - p_2$ (standard error could be from MLE or nonparametric plug-in estimate, probably the former is better since the MLE is asymptotically optimal).
- Could use the t-test to compare the means (estimated $\hat p$ in this case) of each group with the placebo group. More precise for small samples.
- Exact tests would be binoimial test or Fisher exact test.
- Likelihood ratio test for $\theta = p_1 - p_2$.

Estimated odds are $\hat p_1 / (1 - \hat p_1)$ and $\hat p_2 / (1 - \hat p_2)$. The odds ratio is simply the ratio of those two estimates.

In [13]:
# Data

df = pd.DataFrame(
    {'Number': [80, 75, 85, 67, 85],
    'Incidence': [45, 26, 52, 35, 37]
    },
    index = ['Placebo', 'Chlorpromazine', 'Dimenhydrate', 'Pentobarbital (100 mg)', 'Pentobarbital (150 mg)'])

In [14]:
# Wald test

df['p hat'] = df['Incidence'] / df['Number']
df['var'] = df['p hat'] * (1 - df['p hat']) / df['Number']

df['theta hat'] = df.loc['Placebo']['p hat'] - df['p hat']
df['se'] = np.sqrt(df['var']  + df.loc['Placebo']['var'])

df['W'] = np.abs(df['theta hat'] / df['se'])
df['p-value'] = 2 * st.norm.cdf(-df['W'])

df['odds'] = df['p hat'] / (1 - df['p hat'])
df['odds ratio'] = df['odds'] / df.loc['Placebo']['odds']

df[['theta hat','odds ratio', 'p-value']][df.index != 'Placebo']

,theta hat,odds ratio,p-value
Chlorpromazine,0.215833,0.412698,0.005703
Dimenhydrate,-0.049265,1.225589,0.520232
Pentobarbital (100 mg),0.040112,0.850694,0.626664
Pentobarbital (150 mg),0.127206,0.599537,0.099639


At a 5% level only the Chlorpromazine shows a significant difference. The odds ratio is 0.4, which means that the odds are estimated at double those of the placebo!

Now we apply the Bonferroni Method and the Benjamini-Hochberg Method (aka FDR method).

Because we did four tests, the Bonferroni method divides the level by 4. Even then Chlorpromazine shows a significant difference, since the p-value is $0.006$, which is less than $0.05/4$.

Benjamini-Hochberg:
- Order the p-values 0.006, 0.1, 0.5, 0.6
- Find greatest $P_{(i)}$ which is less than $i/4\cdot0.05$, in this case $0.006$
- Reject the hypotheses for $P_{(i)}$ and all p-values below and fail to reject the rest.

We can adjusst the p-values with the Bonferroni method, by multiplying by the number of tests, in this case $4$. The BH method also gives a way to adjust the p-values. The p-value 0.006 would be adjusted to $0.006 \cdot 4/1 = 0.0015$, which happens to be the same as the Bonferroni adjustment.

With both methods, Chlorpromazine shows a significant effect, with adjusted p-value $0.0015$.

**Problem 12** Let $X_1,\dots,X_n \sim \text{Poisson}(\lambda)$.

(a) Let $\lambda_0 > 0$. Find the size $\alpha$ Wald test for $H_0:\lambda=\lambda_0$ versus $H_1:\lambda\ne\lambda_0$.

(b) Let $\lambda_0=1$, $n=20$ and $\alpha=.05$. Simulate $X_1,\dots,X_n \sim \text{Poisson}(\lambda_0)$ ans perform the Wald test. Repeat many times and count how often you reject the null. How close is the type I error rate to 0.05?

Part (a):

From Exercise 5 in Chapter 9: $\hat\lambda = \bar X_n$ and $\hat{\text{se}}  =  \sqrt{\hat\lambda/n} =  \sqrt{\bar X_n/n}$.

The size is given by $$\mathbb{P}_{\lambda_0}\left(\left|\frac{\hat\lambda - \lambda_0}{\hat{\text{se}}}\right| \ge z_{\alpha/2}\right) = \mathbb{P}_{\lambda_0}\left(\left|\frac{\bar X_n - \lambda_0}{\sqrt{\bar X_n/n}}\right| \ge z_{\alpha/2}\right)$$

In [15]:
lambda0 = 1.
size = 20
alpha = 0.05
rv = st.poisson(lambda0)
z = st.norm.ppf(1 - alpha/2)
reps =  10**6

sample  = rv.rvs(size=size*reps).reshape(reps, size)
lambda_hat = sample.mean(axis=1)
rejections = np.sum(np.abs((lambda_hat - lambda0) * np.sqrt(size / lambda_hat)) > z)

rejections / reps

0.052652